# import packages

In [ ]:
import pandas as pd
import numpy as np
import pandas_profiling
import imblearn
import openpyxl
import time
from sklearn.preprocessing import Imputer

# only remain the records with on missing value on edss  

In [5]:
# create list of features with time appended
# e.g. create_list('race', 5) will create a list of 5 elements [race_0, race_6, race_12, race_18, race_24]
def create_list(feat, num=4):
    list = []
    for i in range(1,num):
        list.append(feat + "_" + str(i))
    return list

# interpolated Nan values of edsses
# numYear to interpolate (5 or 10 years)
def interpolate_edss(df, numYear):
    edss_y=create_list('EDSS', 2*(numYear+2))#这个地方看看怎么改
    interpolated_edss = df[edss_y].interpolate()
    interpolated_df = df.copy()
    interpolated_df[edss_y] = interpolated_edss
    return interpolated_edss, interpolated_df
raw_data = pd.read_csv('newone.csv', index_col=None, na_values=['NA'])
df=raw_data.copy()
# index of rows having nan values at column EDSS_60
index_60 = raw_data['EDSS_6'].index[raw_data['EDSS_6'].apply(np.isnan)].tolist()
ddf=df.drop(index_60)
index_1 = ddf['EDSS_1'].index[ddf['EDSS_1'].apply(np.isnan)].tolist()
index_2 = ddf['EDSS_2'].index[ddf['EDSS_2'].apply(np.isnan)].tolist()
index_3 = ddf['EDSS_3'].index[ddf['EDSS_3'].apply(np.isnan)].tolist()
drpset = set(index_1).intersection(index_2).intersection(index_3)
ddf = ddf.drop(drpset)
ddf.to_csv('newdata400.csv', na_rep = 'NA', index=False, encoding='utf-8')

# process the all variables and add differences between differnt timesteps

In [ ]:
#define functions
def get_idx(hdf, feat, switch=True):
    df2 = hdf[feat].copy()
    # replace X with np.nan
    if switch:
        df2.replace({'X' : np.nan}, inplace = True)
        df2.replace({'x' : np.nan}, inplace = True)
    # drop rows having all Nan values
    idx2 = (df2.dropna(axis=0, how='all').index.tolist())
    df2 = df2.astype(float)
    dfr = df2.loc[idx2].interpolate(axis=1, limit_direction='both')
    return dfr

# function to binarize a longitudinal dataframe
# transpose, flatten, catergorize, factorize, reshape back to matrix
def cat_factorize(df, bins=5):
    flatten = df.values.T.reshape(-1,)
    tranform = pd.factorize(pd.cut(flatten, bins),sort=True)[0]
    toMatrix = tranform.reshape(df.shape[1], df.shape[0]).T
    return pd.DataFrame(data=toMatrix, columns=df.columns)

def get_idx_N(hdf, feat, repl, switch=True):
    df2 = hdf[feat].copy()
    # replace X with np.nan
    if switch:
        df2.replace(repl, inplace = True)
    # drop rows having all Nan values
    idx2 = (df2.dropna(axis=0, how='all').index.tolist())
    dfr = df2.loc[idx2].interpolate(axis=1, limit_direction='both')
    return dfr

In [22]:
# create a group for each temporal feature
Pyramidal = create_list("Pyramidal")
Cerebellar = create_list("Cerebellar")
Brainstem = create_list("Brainstem")
Bowel_bladder_raw = create_list("Bowel_bladder_raw")
MENTAL_FUNCTION = create_list("MENTAL_FUNCTION")
Visual_raw=create_list("Visual_raw")
Sensory=create_list("Sensory")
Timed_Walk_Trial_1_time=create_list("Timed_Walk_Trial_1_time")
Timed_Walk_Trial_2_time=create_list("Timed_Walk_Trial_2_time")
LESION_VOLUME=create_list("LESION_VOLUME")
Brain_white_matter_volume=create_list("Brain_white_matter_volume")
Brain_grey_matter_volume=create_list("Brain_grey_matter_volume")
Brain_volume=create_list("Brain_volume")
Cortical_grey_matter_volume=create_list("Cortical_grey_matter_volume")
Ventricular_CSF_volume=create_list("Ventricular_CSF_volume")
TOTAL_GD=create_list("TOTAL_GD")
attackprev180days=create_list("attackprev180days")
attackprev2y=create_list("attackprev2y")
ontreat=create_list("ontreat")
timeontreat_days=create_list("timeontreat_days")
EDSS=create_list("EDSS")

#process the category data
# continuous features with sensory function
dmgr_cols = [ 'SEX', 'RACE_DESC', 'ETHNICITY_DESC']
age_col = ['visit_age_1']

file_path = "newdata400.csv"
whole_df = pd.read_csv(file_path, index_col=None) 

In [17]:
#add the differences
hdf = whole_df.copy()
df1 = get_idx(hdf, Pyramidal,True  )    
df2 = get_idx(hdf, Cerebellar,)    
df3 = get_idx(hdf, Brainstem, True )    
df5 = get_idx(hdf, Cerebellar, )    
df6 = get_idx(hdf, Bowel_bladder_raw, False) 
df8 = get_idx(hdf, MENTAL_FUNCTION, False)
df11= get_idx(hdf, Sensory,True  )  
df12 = get_idx(hdf, Timed_Walk_Trial_2_time,False  )  
df13 = get_idx(hdf, Brain_white_matter_volume,False  )  
df14 = get_idx(hdf, Brain_grey_matter_volume,False  )  
df15 = get_idx(hdf, Cortical_grey_matter_volume,False  )  
df16 = get_idx(hdf, Cortical_grey_matter_volume,False  )  
df17 = get_idx(hdf, Ventricular_CSF_volume,False  )  
df18 = get_idx(hdf, TOTAL_GD,False  )  
df19 = get_idx(hdf, attackprev180days,False  )  
df20 = get_idx(hdf, attackprev2y,False  )  
df21 = get_idx(hdf, ontreat,False  )  
df22 = get_idx(hdf, timeontreat_days,False  )  
for df in (df1,df2,df3,df5,df6,df8):
    df[df.columns[0][:-2]+'_' +'difference1']=df.iloc[:,1]-df.iloc[:,0]
    df[df.columns[0][:-2]+'_' 'difference2']=df.iloc[:,2]-df.iloc[:,1]
cclist = []
for df in [df1, df2, df3, df5, df6,  df8]:
    cclist.append(df)
# dataframe of all functions
sfdf = pd.concat(cclist, axis=1)
df9_edss = get_idx(hdf, EDSS, False)
df9_edss['edss_difference1']=df9_edss.iloc[:,1]-df9_edss.iloc[:,0]
df9_edss['edss_difference2']=df9_edss.iloc[:,2]-df9_edss.iloc[:,1]
age_col_cf = cat_factorize(hdf[age_col], 3)
for df in (df11, df12, df13, df14, df15, df16, df17, df18, df19, df20, df21, df22):
    df[df.columns[0][:-2]+'_'+'difference1']=df.iloc[:,1]-df.iloc[:,0]
    df[df.columns[0][:-2]+'_'+'difference2']=df.iloc[:,2]-df.iloc[:,1]
bblist = []
for df in [df11, df12, df13, df14, df15, df16, df17, df18, df19, df20, df21, df22]:
    bblist.append(df)
    # dataframe of all functions
bbdf = pd.concat(bblist, axis=1)

ttlist = []
for df in [hdf[dmgr_cols], age_col_cf, sfdf, bbdf, df9_edss]:
    ttlist.append(df)
ttdf = pd.concat(ttlist, axis=1)
idx60 = list(ddf.index)
ppdts = ttdf.loc[idx60]
labels = hdf.loc[idx60]['EDSS_6']-hdf.loc[idx60]['EDSS_1']

# save the processed independent variables and lables

In [41]:
diff = 1.5
saved_labels = labels.copy()
saved_labels[labels>=diff] = 1
saved_labels[labels<diff] = 0
pd.DataFrame(saved_labels, columns=['labels']).astype(int).to_csv('Ys.csv', na_rep = 'NA', index=False, columns=['labels'], encoding='utf-8')
new_X_fn = ppdts.fillna(-1)
dm_X = pd.get_dummies(new_X_fn)
dm_X.to_csv('new_D1_X.csv', na_rep = 'NA', index=False, encoding='utf-8')